# word vector
## I. representation learning
1. representation learning \
word vector是representation learning的一种。通过self-supervised方法来学习inputs的representation，而不是手动通过feature engineering来创建representation of inputs。

2. model of word meaning: 一个好的model of word meaning应该可以较好的表达词汇的含义在语义空间的分布。比如能够表达：\
① 近义词、反义词。 \
② 词语的感情色彩：褒义和贬义 \
③ 同属于一簇的词汇（如：buy，sell，pay都是购物场景的词汇；cat，dog都是动物）\
更一般地说，一个好的model of word meaning应该有助于解决meaning-related task。

3. lexical semantics（词汇语义） \
⑴ lemma和wordform：lemma是词元、词根，wordform是用词根生成的具体的词形。每个lemma有多种含义（word sense）\
⑵ synonyms同义词: \
① synonym between word senses：指1个word的1种word sense与另一个word的1种word sense相同。这是可能发生的。\
② synonym between words：指两个word含义相同可以互相替代。<font color=red>但实际上没有真正的synonym between words，water和$H_2O$指同样的东西，但使用场景不同。</font> \
<font color=green>**在representation learning中通常关注的是word similarity**：在word sense的层面建模的话，同一个word有不同的representation，实际任务中很难处理。如果改在word层面建模，因为没有真正的word synonym，关注synonym没有意义，所以关注word similarity是更符合word真实情况的方式。</font>\
⑶ word relatedness: 词与词之间的关系不止similarity。还有很多其他关系，如：\
① sematic field：一种典型的词之间的relatedness是<font color=blue>**词语有相同的sematic field。**</font>比如：waiter,menu,plate,food, chef都属于sematic field of restaurant，而且这些词本身的词义没有similarity。\
② sematic frame：一组含有特定事件（event）信息的词，不同的词在事件中的角色不同。比如，以transaction这个event为例，会有seller, buyer, pay, money, goods等角色，而不同的words会承担这些角色。如果representation可以表达出这部分内容的话，就能识别出Bob bought a desk from Jim. Jim sold his desk to Bob.这两句话含义相同。\
③ connotation: 语言中的情绪，包括：emotion, sentiment, evaluation, or opinion. \
④ 其他，反义词、上义词、下义词

4. vector sematics \
⑴ 含义：指的是用vector表示word，每个word是多维语义空间中的一个点。这个语义空间是利用语料中的word disrtibution信息得到的。 \
⑵ 优点：可以用无监督学习自动从语料中获得vector sematic \
⑶ 方法：两种最基础的方法是tf-idf和ppmi

## II. co-occurrence matrix
### II.1 term-document matrix
1. 结构: 每行表示词汇表中的1个word，每列对应corpus中的1个document，table中第i行j列的value是：$$v_{i,j}=\# word_i\ in\ doc_j $$
表示word i在document j中出现的次数。word对应的row value作为该word的vector。
2. intuition： \
① 用document中包含的word信息来表达document。 \
② 如果两个doc中出现的word相似，那么这两个doc的内容会更相似。可以用来做信息检索。\
③ 用word出现的document信息来表达word meaning。 \
④ <font color=red>如果两个word出现在类似的doc中，说明这两个word相似度高。</font>
3. vector的数值特点：sparse

### II.2 term-term matrix (或word-word matrix)
1. 结构: 每行每列都对应词汇表中的1个word，table中第i行j列的value是：$$v_{i,j}=\# word_j\ in\ window\ of\ word_i $$
行中的word是target word，列中的word是context word。table中第i行j列的值是整个语料中，context word出现在target word的context window中的次数。
2. intuition： \
① 用word上下文窗口中的其他word信息，也即co-occurrence count来表达word meaning。 \
② <font color=red>如果两个word总是出现在彼此的context window中，对应位置的co-occurrence count value大，说明这两个word关系紧密。</font>
3. vector的数值特点：sparse

### II.3 measuring similarity and relatedness
#### 1. inner product
⑴ 已知documents的数量有N个，每个word用N维向量表示，两个word v和w之间的相似性可以用他们vector的inner product来衡量：
$$<v, w>= {\textstyle \sum_{i=1}^{N}}v_iw_i$$
当vector来自term-document matrix时，N=|D|；来自term-term matrix时，N=|V|。\
⑵ 问题：如果vector中非零元素的个数越多，那么inner product的结果就越大。这意味着：
① term-document matrix中，常见高频词汇（the, it等）出现的doc多，非零元素多，因此他与其他word的inner product值偏大。\
② term-term matrix中，高频词汇与更多词汇有co-occurrence，且总的来说与单个context word的co-occurrence次数也多，所以inner product偏大。\
上面两种情况并不表示较高的similarity或者relatedness，这些高频词汇提供的信息本身也有限。
#### 2. cosine
⑴ 针对高频词汇带来的similarity度量不准确的问题，可以对inner product做normalization：$$\frac{v·w}{|v||w|}=\cos \theta$$
⑵ 含义：此时向量之间的夹角表示两个word的relatedness \
⑶ <font color=red>[rk's note]这里修正的方法也许可以有其他方式。比如不用向量单位化$v^{'}=\frac{v}{|v|}$，而用$v^{'}=\frac{v}{ {\textstyle \sum_{i=1}^{N}} v_i}$</font>

### II.4 对co-occurrence matrix做权重修正
1. **背景**：简单的co-occurrence count无法很好地度量words之间的relatedness，一个很大的问题还是来自常见高频词。\
① 在term-document matrix中，如果word经常出现在某个doc中，但不经常出现在别的doc中，这个word对于标记该doc会很重要；如果word经常出现在所有doc中，那么它对于标记doc就不重要。\
② 在term-term matrix中，如果word经常出现在特定word的context window中，但不是经常出现在大量的words的context window中，那么前一种word所表达的信息就比后一种更多。
2. <font color=green>**solution**：给matrix中的value加上weights，让信息量大的words的weights大，信息量小的常见高频词的weights小，用weights修正有信息词和无信息高频词的vector value，实现两种情况的区分。\
tf-idf weights就是处理term-document matrix的weights，ppmi则是处理term-term matrix的weights。

#### II.4.1 TF-IDF model
1. 方法 
$$\begin{align} 
  w_{t,d} & = tf_{t, d}*idf_t \\
\\
其中， tf_{t, d} & = log_{10}(count(t,d)+1)\\
idf_t & = log_{10}(\frac{N}{df_t})\\
 df_t & =  {\textstyle \sum_{i=1}^{N}}\#1\{word_t\ in\ doc_i\} 
\end{align}$$
2. 思路 \
⑴ <font color=blue>$tf_{t,d}$是term frequency of word t in document d.</font>\
① 不用raw frequency，因为出现100次的word对doc的重要性不太可能是出现1次的词的100倍，所以用$log_{10}$来降低数量级。\
② 加1是为了避免log0 \
⑵ <font color=blue>$idf_t$是inverse document frequency。</font> \
① 如果一个词出现在很多doc中，那么它的idf就会更低。\
② 由于N是doc的数量，这个值通常都很大，所以也用log来降低数量级
3. 应用场景：信息检索中用的多

#### II.4.2 PPMI (positive pointwise mutual information)
1. 直觉：衡量两个word之间相关性的最好方式是，先计算假如完全是by chance的话，两个word的co-occurrence的概率，然后看他们实际co-occurrence的概率。如果超过by chance的话，那么他们就相关，超过越多，相关性越强。

2. **pointwise mutual information,PMI**：两个word实际co-occurrence的概率与假设他们相互独立时，co-occurrence的概率之比。用于衡量两个word之间的相关性。
⑴ 定义
$$I(x, y)=log_2\frac{P(x, y)}{P(x)P(y)}
\ \Rightarrow \begin{cases}
 =\frac{1}{\alpha } >0 & \text{ if } 两个词总是同时出现，单个词的概率为\alpha \\
 =0 & \text{ if } 两个词独立，没有关联 \\
 <0 & \text{ if } 两个词很少同时出现 \\
\rightarrow -\infty  & \text{ if } 两个词几乎从来不同时出现
\end{cases}$$
⑵ 问题：PMI的值为负的时候，通常不可靠，除非语料的长度趋于无穷大。 \
假如单个word发生的概率是$10^{-6}$，那么他们独立时$P(x, y)=10^{-12}$。如果co-occurrence的概率比这个概率更低，比如$P(x, y)=10^{-13}$，这就意味着，corpus的长度至少要10万亿以上长度的corpus。\
⑶ solution: 使用Positive PMI

3. **positive PMI**: \
⑴ 定义
$$PPMI(x, y)=max\left ( log_2\frac{P(x, y)}{P(x)P(y)}, 0 \right ) $$
⑵ 问题：<font color=red>biased toward infrequent events.出现频次很低的word倾向于有较高的PPMI value </font> \
· 原因分析：如前面计算公式中，如果α很小，那么1/α就会很大。 \
⑶ solution: 改变context word的probability计算方式，认为修正结果:
$$\begin{align} 
  PPMI(x, y)& =max\left ( log_2\frac{P(x, y)}{P(x)P_\alpha (y)}, 0 \right ) \\
P_\alpha (y)& =\frac{count(y)^\alpha }{ {\textstyle \sum_{y}^{}}count(y)^\alpha  } 
\end{align}$$

#### II.4.3 weights的用法
将他们作为weights与co-occurrence matrix中的value相乘得到新的co-occurrence matrix。可以用cosine of new vector来衡量similarity or relatedness.
1. tf-idf经常用于判断两个doc是不是相似
2. tf-idf和PPMI都可以用于判断两个word的相似性

## III. word2vec
<font color=green>**成果**：这篇论文的提出，解决了在billion级的语料，million级的vector长度上做word representation的问题。在它之前的那些模型都只能a few hundred of millions数量的语料，vector长度在50-100的范围。</font>
#### 和co-occurrence matrix的比较
1. 数值特征的差异：word2vec得到的vector是dense的，并且vector的每个元素不再像co-occurrence matrix中那样有明确的含义。
2. 实践效果：word2vec在实际任务上的效果往往比co-occurrence matrix中vector的效果好。原因：\
① 因为维度少，要learn的参数少，overfit少\
② wrod2ec 比co-occurrence matrix能更好地表达近义关系。比如：在word-word vector中，column中的‘car’和‘automobile’对应不同的维度，他们在数值上看，是两个正交的方向，两个vector的context中分别有car和automobile的话，这两个信息无法交互。

#### 包含的模型：
CBOW，SG，SGNS
#### 模型的共性特点：
1. <font color=blue>model思路：训练一个classifier，判断word是不是target word的context word。但是训练出来之后并不记录结果，而是将classifier的weight parameters作为target word和context word的word vector。</font>
2. **价值点：可以做到self-supervised learning**

3. intuition：用inner product衡量relatedness或者similarity，如果两个word的relatedness越高，那么他们互相在彼此的上下文中的概率就越大。$$Similarity(w,c)\approx \left \langle c,w \right \rangle$$

4. 符号：\
① vocabulary的长度为N，corpus的长度为T \
② context window长度为L \
③ 1个positive sample(target word and context word pair)对应的negative sample(target word and non-context word pair)数量为k \
④ $P(c|w)$表示word $c$是word w的context word的概率 \
⑤ $P(c_{1:L}|w)$表示$c_{1:L}$都是word w的context words的概率

5. 表达intuition的方式：用logistic或者sigmoid function，将similarity与probability联系起来。\
⑴ 如果用logistic \
$$P(c_i|w)=\sigma (\left \langle c_i,w \right \rangle)=\frac{1}{1+e^{-\left \langle c_i,w \right \rangle}}$$
⑵ 如果用sigmoid \
$$P(c_i|w)=\frac{e^{c_i·w}}{ {\textstyle \sum_{c_j\in |v|}^{}}e^{c_j·w} }$$
**[rk's note]** \
① 最初word2vec中的SG model用的是softmax，但是计算复杂度太高。所以实际算法中用的是hierarchical softmax。\
② <font color=red>**SGNS为了减少运算复杂度，改用logistic，也因此要配合用negative sample**</font> \
<font color=green>如果用sigmoid的话，应该不需要negative sample。因为sigmoid中计算P(c|w)的方式已经包含了如果要最大化$P(c_i|w)$那么其他非$c_i$的context的概率就会降低，sigmoid中normalize的方式已经暗含了这个约束。但是logistic中没有。</font> \
实际效果看SGNS不仅提高了运算效率，而且如果用offset method来衡量的similarity作为评价标准的话，SGNS得到的word representation更好。

### III.2 skip-gram
1. 思路：\
⑴ 给一个target word，用它计算所选窗口中context word发生的概率是:$$P(c_i|w)=\frac{e^{c_i·w}}{ {\textstyle \sum_{c_j\in |v|}^{}}e^{c_j·w} }$$
⑵ 简化假设： \
① context word相互独立，所有L个context word一起出现的概率：$$P(c_{1:L}|w) = {\textstyle \prod_{i=1}^{L}} P(c_i|w)$$
<font color=red>这个设定也导致SG学习到的word representation无法表达order，比如：习语短语如Boston Dynamic，无法与Dynamic Boston做区分，而且本身表达的含义不应该是"Boston + Dynamic"。</font> \
② 为简化求解过程，让每个word作为target word和context word有不同的vector。 \
⑶ 目标是最大化语料库中样本window中target、context word样本发生的概率：$$
L = \frac{1}{T} {\textstyle \sum_{t=1}^{T}}  {\textstyle \sum_{j=1}^{L}}logP(c_j|w_t) 
$$
2. 特点：\
<font color=blue>① 模型简单运算效率高，加上self-supervised learning可以利用大量的预料，因此把实践中word vector训练的预料规模和vector的长度都大幅提升。\
② 不足是学习到的word representation无法表达order </font>

### III.2 skip-gram with negative sampling (SGNS)
1. **在SG基础上做了3点改进** \
<font color=blue>① 用sub-sampling of frequent words，优点是提高训练效率，增加低频词汇的表达准确度 \
② 用negative sampling，优点是提高训练效率，增加高频词汇的表达准确度 \
③ 对习语做专门的训练</font>

2. 模型设定： \
⑴ 简化假设context word相互独立，所以有$$\begin{align} 
P(c_{1:L}|w) & = {\textstyle \prod_{i=1}^{L}} P(c_i|w) \\
& = {\textstyle \prod_{i=1}^{L}} \sigma (\left \langle c_i,w \right \rangle) \\
& ={\textstyle \prod_{i=1}^{L}}\frac{1}{1+e^{-\left \langle c_i,w \right \rangle}} \\
\end{align}$$ 
<font color=blue>虽然这个独立假设和SG中一样，会导致无法表达短语，但SGNS paper中用了phrase vector来解决短语识别问题。</font><font color=red>SGNS并没有从根本上解决习语的表达的问题，只是单独为习语短语(idiomatic phrase)如"New York"做了训练。使得"New York"有自己的vector，而不是用"New"和"York"的vector来表达。</font>\
⑵ 为了简化求解过程，让每个word作为target word和context word有不同的vector。 \
⑶ 模型的目标是找到合适的vector，使得语料库中互为context的words的$P(c|w)$最大化，不是context的words的$P(c|w)$最小化。

3. learning algorithm \
⑴ 抽样：\
① 从语料中抽target word，得到长度为L的窗口内的L个context word，构成L个positive sample。\
② 一组样本包含1个positive sample和与之匹配的k个negative sample。negative sample是用target word做均匀抽样得到的。但该均匀抽样使用的概率不是raw frequency，而是weighted unigram frequency。[见下面说明] \
⑵ 整体目标是：$$\begin{align} 
  L  =  \frac{1}{T}{\textstyle \prod_{t=1}^{T}}  {\textstyle \prod_{i=1}^{N}}P(c_{pos_i}|w_t) {\textstyle \prod_{j=1}^{k}}(1-P(c_{neg_j}|w_t))  \\
\\
\end{align}$$
⑶ 实际用SGD做迭代：
一次迭代（对应一组样本）的最大似然估计： \
$$\begin{align} 
\max L(w, c_{pos},c_{neg_i}) & = \max P(c_{pos|w}) {\textstyle \prod_{i=1}^{k}} (1-P(c_{neg_i}|w)) \\
& = \max log\left ( P(c_{pos|w}) {\textstyle \prod_{i=1}^{k}} (1-P(c_{neg_i}|w)) \right ) \\
& = \max logP(c_{pos|w}) +  {\textstyle \sum_{i=1}^{k}} log\left (1-P(c_{neg_i}|w)  \right ) \\
& = \max log\sigma (c_{pos}·w) + {\textstyle \sum_{i=1}^{k}} log \sigma (-c_{neg_i}·w ) \\
& = \min log(1+e^{c_{pos}·w}) + {\textstyle \sum_{i=1}^{k}}log(1+e^{-c_{neg_i}·w})
\end{align}$$
梯度：$$\begin{align} 
\frac{\partial L}{\partial c_{pos}} & =[\sigma (c_{pos}·w)-1]w \\
\frac{\partial L}{\partial c_{neg_i}} & =\sigma (c_{neg_i}·w)w \\
\frac{\partial L}{\partial w} & =[\sigma (c_{pos}·w)-1]c_{pos}+ {\textstyle \sum_{i=1}^{k}}\sigma (c_{neg_i}·w)c_{neg_i}  \\
\end{align}
$$

4. negative sample方法 \
negative sample的抽样不是按照word的frequency来做均匀抽样，而是要加权重，用weighted unigram frequency。<font color=blue>主要是为了让低频词被抽中的概率高一些，高频词的低一些。因为**词语在预料中的分布是长尾分布。**</font>
$$P_\alpha(w)=\frac{count(w)^\alpha}{ {\textstyle \sum_{w^{'}}^{}}count(w^{'})^\alpha  
},\ 经验取\alpha=0.75$$

5. 应用 \
因为learn到的结果里面每个word有两个vector，在具体使用的时候，通常是取两个vector的均值，或者删除作为contect word得到的context vector，只用target vector。

### III.3 GloVe

## IV. word embedding的可视化
方法：
1. hierachical clustering
2. t-SNE

## V. sematic properties of embeddings
1. context window长度设定会让embeddings获得三种不同的similarity or association：\
一般k的长度会选两边各1-10个words，k=2~20。  \
① 如果k较小，就是周围2、3个位置的词，得到的word representation中带有的syntactic（句法的）信息更多 \
② 如果k虽然还是小，但不是只看周围2、3个位置，那么这些位置的context word与target word的相似性相对大。 \
③ 如果k较大，那么positive sample中的context word与target word的关系是topically related，但similarity不见得大。

2. 区分两种similarity or relatedness \
① first-order co-occurrence, 又称为syntagmatic association（组合相关性），表示两个词通常出现在彼此的旁边。比如wrote和book，ride和bike。 \
② second-order co-occurrence, 又称为paradigmatic association（排比关系），表示两个词有相同的邻近词。比如wrote，said和remarked。

3. embeddings捕捉语义关系的能力: \
⑴ analogy meaning \
analogy problem的问题形式: a is to b as $a^{'}$ is to what? \
比如: $apple:tree::grape:\_?$ 答案是vine.  \
⑵ 用word vector来对类比关系进行数学化处理的方法是parallelogram method：\
① 先定义distance function。\
② 再求解：$$b^{'} = \underset{x}{argmin}\ distance(x, b-a+a^{'})$$
⑶ 实践结果：word2vec和Glove这些方法在特定的analogy problem上效果很好。但通常是distance较小，涉及的word是高频词汇，涉及的词间关系也是特定关系如（性别职业、国家首都、动词的名词对应等），超过这些特定范围的效果就不太好。\
⑷ <font color=green>有的研究者认为，问题在于使用$b^{'} = \underset{x}{argmin}\ distance(x, b-a+a^{'})$ 的parallelogram method过于简化，不能反映出word之间的复杂关系。</font>

4. sematic meaning的变化  \
现实中，词语的sematic meaning在长期来看是不断变化的。如果使用不同时代的预料来得到word embedding，得到的结果会不一样。

## VI. evaluating vector models
1. extrinsic evaluation：一般来说最好的评价方式就是看在具体任务上的结果
2. intrinsic evaluation：最常用的方式是test the performance on similarity。比较人对词语相似性的评估和embeding的评估结果。